In [ ]:
import pandas as pd
import pyotp
from datetime import datetime
import time
import csv
import logging
import websocket as ws_client
import threading

from NorenRestApiPy.NorenApi import NorenApi

class ShoonyaApiPy(NorenApi):
    def __init__(self):
        NorenApi.__init__(self, host='https://api.shoonya.com/NorenWClientTP/', websocket='wss://api.shoonya.com/NorenWSTP/')        
        global api
        api = self

import logging
logging.basicConfig(level=logging.DEBUG)

usercred = pd.read_excel("usercred.xlsx")
user    = usercred.iloc[0,0]
pwd     = usercred.iloc[0,1]
vc      = usercred.iloc[0,2]
app_key = usercred.iloc[0,3]
imei    = usercred.iloc[0,4]
qr = usercred.iloc[0,5]
factor2 = pyotp.TOTP(qr).now()

api = ShoonyaApiPy()
ret = api.login(userid=user, password=pwd, twoFA=factor2, vendor_code=vc, api_secret=app_key, imei=imei)

In [ ]:
#####Getting historical data
def get_time(time_string):
    """
    Converts a timestamp string to a Unix timestamp (seconds since epoch).
    """
    data = time.strptime(time_string, '%Y-%m-%d %H:%M:%S')  # Parse time string
    return int(time.mktime(data))  # Convert to Unix timestamp

start_time = "2023-01-01 09:15:00"
end_time = "2024-06-22 15:15:00"

st = get_time(start_time)
et = get_time(end_time)


hist_bn_data_1 = pd.DataFrame(api.get_time_price_series(exchange="NSE",token='26009', starttime=st, endtime=et, interval=1))
hist_nifty_data_1 = pd.DataFrame(api.get_time_price_series(exchange="NSE",token='26000', starttime=st, endtime=et, interval=1))
hist_bn_data_5 = pd.DataFrame(api.get_time_price_series(exchange="NSE",token='26009', starttime=st, endtime=et, interval=5))
hist_nifty_data_5 = pd.DataFrame(api.get_time_price_series(exchange="NSE",token='26000', starttime=st, endtime=et, interval=5))


# Convert 'time' column to datetime
hist_bn_data_1['time'] = pd.to_datetime(hist_bn_data_1['time'])
hist_nifty_data_1['time'] = pd.to_datetime(hist_nifty_data_1['time'])
hist_bn_data_5['time'] = pd.to_datetime(hist_bn_data_5['time'])
hist_nifty_data_5['time'] = pd.to_datetime(hist_nifty_data_5['time'])

# Convert numerical columns to float with two decimal places
numeric_columns = ['into', 'inth', 'intl', 'intc']
for col in numeric_columns:
    hist_bn_data_1[col] = pd.to_numeric(hist_bn_data_1[col], errors='coerce').round(2)  # Convert to numeric first, then round
    hist_nifty_data_1[col] = pd.to_numeric(hist_nifty_data_1[col], errors='coerce').round(2)
    hist_bn_data_5[col] = pd.to_numeric(hist_bn_data_5[col], errors='coerce').round(2)
    hist_nifty_data_5[col] = pd.to_numeric(hist_nifty_data_5[col], errors='coerce').round(2)

In [ ]:
hist_bn_data_1_reversed = hist_bn_data_1[::-1]
hist_nifty_data_1_reversed = hist_nifty_data_1[::-1]
hist_bn_data_5_reversed = hist_bn_data_5[::-1]
hist_nifty_data_5_reversed = hist_nifty_data_5[::-1]


# Set timeframe column as index (replace 'timeframe_column' with your actual column name)
dataframes = {
    "hist_bn_data_1_reversed": hist_bn_data_1_reversed,
    "hist_nifty_data_1_reversed": hist_nifty_data_1_reversed,
    "hist_bn_data_5_reversed": hist_bn_data_5_reversed,
    "hist_nifty_data_5_reversed": hist_nifty_data_5_reversed,
}

for df_name, df in dataframes.items():
    df.set_index('time', inplace=True)



In [ ]:
# import pandas as pd
# import pandas_ta as ta

# hist_bn_data_1_reversed.loc[:, 'alma1'] = ta.alma(hist_bn_data_1_reversed['intc'], length=10, sigma=6, distribution_offset=0.85)
# hist_nifty_data_1_reversed.loc[:, 'alma1'] = ta.alma(hist_nifty_data_1_reversed['intc'], length=10, sigma=6, distribution_offset=0.85)
# hist_bn_data_5_reversed.loc[:, 'alma1'] = ta.alma(hist_bn_data_5_reversed['intc'], length=10, sigma=6, distribution_offset=0.85)
# hist_nifty_data_5_reversed.loc[:, 'alma1'] = ta.alma(hist_nifty_data_5_reversed['intc'], length=10, sigma=6, distribution_offset=0.85)

# hist_bn_data_1_reversed.loc[:, 'alma2'] = ta.alma(hist_bn_data_1_reversed['intc'], length=10, sigma=6, distribution_offset=1)
# hist_nifty_data_1_reversed.loc[:, 'alma2'] = ta.alma(hist_nifty_data_1_reversed['intc'], length=10, sigma=6, distribution_offset=1)
# hist_bn_data_5_reversed.loc[:, 'alma2'] = ta.alma(hist_bn_data_5_reversed['intc'], length=10, sigma=6, distribution_offset=1)
# hist_nifty_data_5_reversed.loc[:, 'alma2'] = ta.alma(hist_nifty_data_5_reversed['intc'], length=10, sigma=6, distribution_offset=1)



In [4]:
# Define your list of DataFrames
dataframes = [hist_bn_data_1_reversed, hist_nifty_data_1_reversed, hist_bn_data_5_reversed, hist_nifty_data_5_reversed]

# Iterate through each DataFrame
for df in dataframes:
    # Round columns containing 'ALMA' to 2 decimal places
    for col in df.columns:
        if 'ALMA' in col:
            df[col] = df[col].round(2)

pd.set_option('display.float_format', '{:.2f}'.format)

hist_bn_data_1_reversed.to_csv("hist_bn_data_1.csv")
hist_nifty_data_1_reversed.to_csv("hist_nifty_data_1.csv")

hist_bn_data_5_reversed.to_csv("hist_bn_data_5.csv")
hist_nifty_data_5_reversed.to_csv("hist_nifty_data_5.csv")


In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('hist_bn_data_1.csv')

# Convert the `time` column to datetime
df['time'] = pd.to_datetime(df['time'])

# Filter data between '2024-01-01' and '2024-06-21'
df_filtered = df.loc[(df['time'] >= '2024-01-01') & (df['time'] <= '2024-06-21')].copy()

# Sort the filtered data by `time`
df_filtered = df_filtered.sort_values('time')
df_filtered.to_csv("test.csv")

# Calculate short-term and long-term EMAs
df_filtered['ema_short'] = df_filtered['intc'].ewm(span=5, adjust=False).mean()
df_filtered['ema_long'] = df_filtered['intc'].ewm(span=3, adjust=False).mean()

# Trading time constraints
start_time = pd.Timestamp('09:25:00').time()
end_time = pd.Timestamp('15:15:00').time()
close_time = pd.Timestamp('15:25:00').time()

# Initialize lists to store trade details
entries = []
exits = []
profits = []
trade_types = []

# Variables to track position status
position = None
entry_price = 0
entry_time = None

# Iterate through the dataframe to generate signals and calculate P&L
for i in range(1, len(df_filtered)):
    # Previous EMA
    prev_ema_short = df_filtered["ema_short"].iloc[i - 1]
    prev_ema_long = df_filtered["ema_long"].iloc[i - 1]
    # Current EMA
    curr_ema_short = df_filtered["ema_short"].iloc[i]
    curr_ema_long = df_filtered["ema_long"].iloc[i]
    # Current Close Price
    curr_close = df_filtered["intc"].iloc[i]
    # Current Time
    curr_time = df_filtered['time'].iloc[i]
    curr_time_only = curr_time.time()  # Extract time component
    curr_date = curr_time.date()

    # Skip if time is not within trading hours
    if not (start_time <= curr_time_only <= end_time):
        continue
    
    if curr_date.weekday() != 5 or curr_date.weekday() != 6:
        continue


    # Check for EMA cross above signal (Long signal)
    if prev_ema_long <= prev_ema_short and curr_ema_long > curr_ema_short:
        if position is not None:
            exit_price = curr_close
            exits.append((curr_time, exit_price))
            profits.append(exit_price - entry_price if position == "long" else entry_price - exit_price)
            trade_types.append(position)

        position = "long"
        entry_price = curr_close
        entry_time = curr_time
        entries.append((entry_time, entry_price))

    # Check for EMA cross below signal (Short signal)
    elif prev_ema_long >= prev_ema_short and curr_ema_long < curr_ema_short:
        if position is not None:
            exit_price = curr_close
            exits.append((curr_time, exit_price))
            profits.append(exit_price - entry_price if position == "long" else entry_price - exit_price)
            trade_types.append(position)

        position = "short"
        entry_price = curr_close
        entry_time = curr_time
        entries.append((entry_time, entry_price))

    # Check for exit signal (time exceeding closing time on the same day as entry)
    if position is not None and curr_time_only >= close_time and curr_date == entry_time.date():
        exit_price = curr_close
        exits.append((curr_time, exit_price))
        profits.append(exit_price - entry_price if position == "long" else entry_price - exit_price)
        trade_types.append(position)
        position = None

# Final check for open position at the end of data
if position is not None:
    exit_price = df_filtered['intc'].iloc[-1]
    exits.append((df_filtered['time'].iloc[-1], exit_price))
    profits.append(exit_price - entry_price if position == "long" else entry_price - exit_price)
    trade_types.append(position)

# Combine the results into a dataframe
trades = pd.DataFrame(
    {
        "Entry Time": [e[0] for e in entries],
        "Entry Price": [e[1] for e in entries],
        "Exit Time": [e[0] for e in exits],
        "Exit Price": [e[1] for e in exits],
        "Profit/Loss": profits,
        "Trade Type": trade_types,
    }
)

# Save the trades dataframe to a CSV file
output_file_path = "trade_results_corrected.csv"
trades.to_csv(output_file_path, index=False)

print("Trades results saved to:", output_file_path)


In [ ]:
start_time = pd.Timestamp('15:25:00').time()
start_time

In [ ]:
pip install -U git+https://github.com/twopirllc/pandas-ta.git@development


Entry Time	Entry Price	Exit Time	Exit Price	Profit/Loss	Trade Type
2024-02-02 15:08:00	46017.35	2024-03-02 11:21:00	48650.65	-2633.3	short
